In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/menstrual-health-and-pcod-risk-detection-dataset/period - Copy.csv


In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
df = pd.read_csv("/kaggle/input/menstrual-health-and-pcod-risk-detection-dataset/period - Copy.csv")
df.head()

,number_of_peak,Age,Length_of_cycle,Estimated_day_of_ovulution,Length_of_Leutal_Phase,Length_of_menses,Unusual_Bleeding,Height,Weight,Income,BMI,Mean_of_length_of_cycle,Menses_score
0,3,18,27,14,9,5,no,5 6,76.0,0,26.3,30,4.0
1,4,18,25,17,10,6,yes,5 6,76.0,0,26.3,30,5.0
2,2,19,30,17,13,4,no,5 3,56.0,0,21.9,30,2.0
3,3,19,28,16,14,6,no,5 1,61.0,0,25.4,30,3.0
4,2,19,35,18,15,5,no,5'3,55.0,0,21.5,35,5.0


In [4]:
df.describe()

,number_of_peak,Age,Length_of_cycle,Estimated_day_of_ovulution,Length_of_Leutal_Phase,Length_of_menses,Weight,Income,BMI,Mean_of_length_of_cycle,Menses_score
count,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.0,162.000000,162.000000,162.000000
mean,2.518519,19.333333,29.283951,15.135802,12.753086,5.061728,56.696296,0.0,21.738889,29.320988,3.372840
std,0.706863,1.453204,4.191783,2.352496,3.270538,1.025703,9.471440,0.0,2.975396,5.319196,1.183112
min,1.000000,14.000000,22.000000,7.000000,7.000000,1.000000,40.000000,0.0,16.200000,13.000000,1.000000
25%,2.000000,19.000000,27.000000,14.000000,10.000000,5.000000,51.000000,0.0,20.400000,27.000000,3.000000
50%,2.000000,19.000000,28.000000,15.000000,13.000000,5.000000,55.000000,0.0,21.600000,30.000000,3.000000
75%,3.000000,20.000000,30.000000,16.000000,14.000000,6.000000,60.000000,0.0,22.500000,30.000000,4.000000
max,5.000000,25.000000,40.000000,22.000000,30.000000,8.000000,85.000000,0.0,32.900000,46.000000,5.000000


In [5]:
null_counts = df.isnull().sum()
print("Null counts in each column:")
print(null_counts)

Null counts in each column:
number_of_peak                0
Age                           0
Length_of_cycle               0
Estimated_day_of_ovulution    0
Length_of_Leutal_Phase        0
Length_of_menses              0
Unusual_Bleeding              0
Height                        0
Weight                        0
Income                        0
BMI                           0
Mean_of_length_of_cycle       0
Menses_score                  0
dtype: int64


In [6]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)

Unique values in column 'number_of_peak':
[3 4 2 5 1]
Unique values in column 'Age':
[18 19 20 22 23 14 17 25]
Unique values in column 'Length_of_cycle':
[27 25 30 28 35 40 37 34 26 23 29 24 22 36 38 32]
Unique values in column 'Estimated_day_of_ovulution':
[14 17 16 18 15 10 11 13  7 20 12 22]
Unique values in column 'Length_of_Leutal_Phase':
[ 9 10 13 14 15 12 16 11  7  8 19 30]
Unique values in column 'Length_of_menses':
[5 6 4 3 1 7 8]
Unique values in column 'Unusual_Bleeding':
['no' 'yes' 'No' 'NO']
Unique values in column 'Height':
['5 6' '5 3' '5 1' "5'3" "5'4" "5'1" "5'0" '5\'4"' "5'9" "5'5''" "5'6"
 "5'" "5'2.5" "5'2" "4'8" "4' 11" "5'7" '5 4' "5' 4" "5'5" '56' "5'8"
 '5\'2"' '5.3"' '5\'5"' '5\'1"' '5\'3"']
Unique values in column 'Weight':
[76.  56.  61.  55.  50.  60.  58.  51.  54.  43.  72.  75.  48.  45.5
 53.  47.  82.  57.  40.  64.  55.2 65.  62.  85.  84.  52.  70. ]
Unique values in column 'Income':
[0]
Unique values in column 'BMI':
[26.3  21.9  25.4  21.5  19.   2

In [7]:
df["Unusual_Bleeding"]=df["Unusual_Bleeding"].map({"NO":0,"No":0,"no":0,"yes":1})
df["Menses_score"]=df["Menses_score"].astype(str).map({"1.0":0, "2.0":1, "3.0":2, "4.0":3, "4.2":4, "4.5":5, "5.0":6})

In [8]:
df.head()

,number_of_peak,Age,Length_of_cycle,Estimated_day_of_ovulution,Length_of_Leutal_Phase,Length_of_menses,Unusual_Bleeding,Height,Weight,Income,BMI,Mean_of_length_of_cycle,Menses_score
0,3,18,27,14,9,5,0,5 6,76.0,0,26.3,30,3
1,4,18,25,17,10,6,1,5 6,76.0,0,26.3,30,6
2,2,19,30,17,13,4,0,5 3,56.0,0,21.9,30,1
3,3,19,28,16,14,6,0,5 1,61.0,0,25.4,30,2
4,2,19,35,18,15,5,0,5'3,55.0,0,21.5,35,6


In [9]:
df.drop("Income", axis=1, inplace=True)
df.drop("Height", axis=1, inplace=True)

In [10]:
x=df.drop("Menses_score", axis=1)
y=df["Menses_score"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [11]:
model = xgb.XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss'
)
model.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

In [12]:
y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(classification_report(y_test, y_pred))

Accuracy: 0.8787878787878788
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.75      1.00      0.86         6
           2       1.00      0.85      0.92        13
           3       0.75      0.75      0.75         8
           6       1.00      1.00      1.00         4

    accuracy                           0.88        33
   macro avg       0.90      0.92      0.90        33
weighted avg       0.89      0.88      0.88        33

